### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/302_TROCAS7_Fusion_Apr2021_PEAKS_166/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/302_TROCAS7_Fusion_Apr2021_PEAKS_166


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 302_TROCAS7_Fusion_Apr2021_PEAKS_166_peptide.csv
 302_TROCAS7_Fusion_Apr2021_PEAKS_166_protein.csv
 302_TROCAS7_Fusion_Apr2021_PEAKS_166_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup302 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/302_TROCAS7_Fusion_Apr2021_PEAKS_166/302_TROCAS7_Fusion_Apr2021_PEAKS_166_peptide.csv")

# remove redundant rows
peaksdb302 = pd.DataFrame.drop_duplicates(peaksdbdup302)

print(peaksdb302.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb302.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb302['#Spec']
del peaksdb302['Accession']
del peaksdb302['PTM']
del peaksdb302['AScore']

mean_length = peaksdb302['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup302))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb302))

#look at the dataframe
peaksdb302.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-302-Bay-DUP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-302-Bay-DUP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.306451612903226
# redundant peaksdb peptides in combined dataframe 62
# nonredundant peaksdb peptides in combined dataframe 62


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,90.90,1881.8350,18,1.0,941.9257,90.29,2450000.0,103,14370,20210410_Trocas7_650_Bay302_DDA_120min_2.raw
1,LGEHNIDVLEGNEQFINAAK,78.28,2210.0967,20,0.4,737.7065,93.90,3050000.0,103,15115,20210410_Trocas7_650_Bay302_DDA_120min_2.raw
2,SSGSSYPSLLQC(+57.02)LK,73.99,1525.7446,14,1.1,763.8804,98.64,417000.0,103,16059,20210410_Trocas7_650_Bay302_DDA_120min_2.raw
3,SC(+57.02)AAAGTEC(+57.02)LISGWGN(+.98)TK,49.71,1882.8190,18,2.8,942.4194,91.56,28200.0,103,14631,20210410_Trocas7_650_Bay302_DDA_120min_2.raw
4,LSSPATLNSR,47.62,1044.5564,10,1.0,523.2860,46.21,17800000.0,103,7522,20210410_Trocas7_650_Bay302_DDA_120min_2.raw


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb302['A'] = peaksdb302['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb302['C'] = peaksdb302['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb302['D'] = peaksdb302['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb302['E'] = peaksdb302['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb302['F'] = peaksdb302['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb302['G'] = peaksdb302['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb302['H'] = peaksdb302['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb302 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb302['I'] = peaksdb302['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb302['K'] = peaksdb302['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb302['L'] = peaksdb302['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb302['M'] = peaksdb302['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb302['N'] = peaksdb302['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb302['P'] = peaksdb302['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb302['Q'] = peaksdb302['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb302['R'] = peaksdb302['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb302['S'] = peaksdb302['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb302['T'] = peaksdb302['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb302['V'] = peaksdb302['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb302['W'] = peaksdb302['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb302['Y'] = peaksdb302['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb302['c-carb'] = peaksdb302['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb302['m-oxid'] = peaksdb302['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb302['n-deam'] = peaksdb302['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb302['q-deam'] = peaksdb302['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb302['stripped_peptide'] = peaksdb302['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb302['stripped_length'] = peaksdb302['stripped_peptide'].apply(len)

peaksdb302['NAAF_num.'] = peaksdb302['Area'] / peaksdb302['stripped_length']

# total the number of modifications in sequence
peaksdb302['ptm-total'] = peaksdb302['c-carb'] + peaksdb302['m-oxid'] + peaksdb302['n-deam'] + peaksdb302['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb302['stripped_IL']= peaksdb302['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB.csv")

# check out the results
peaksdb302.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,90.90,1881.8350,18,1.0,941.9257,90.29,2450000.0,103,14370,...,0,2,0,0,0,SCLISGWGNTK,11,2.227273e+05,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,78.28,2210.0967,20,0.4,737.7065,93.90,3050000.0,103,15115,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,1.525000e+05,0,LGEHNLDVLEGNEQFLNAAK
2,SSGSSYPSLLQC(+57.02)LK,73.99,1525.7446,14,1.1,763.8804,98.64,417000.0,103,16059,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,2.978571e+04,1,SSGSSYPSLLQCLK
3,SC(+57.02)AAAGTEC(+57.02)LISGWGN(+.98)TK,49.71,1882.8190,18,2.8,942.4194,91.56,28200.0,103,14631,...,0,2,0,1,0,SCTK,4,7.050000e+03,3,SCTK
4,LSSPATLNSR,47.62,1044.5564,10,1.0,523.2860,46.21,17800000.0,103,7522,...,0,0,0,0,0,LSSPATLNSR,10,1.780000e+06,0,LSSPATLNSR


In [7]:
# keep only stripped peptide I/L and NAAF
db_302 = peaksdb302[['stripped_IL', 'Area', 'NAAF_num.']]

db_302.set_index('stripped_IL')

# write modified dataframe to new txt file
db_302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/302B_BY_T24_GF_PDB_ILnaafs.csv")

db_302.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,2450000.0,2.227273e+05
1,LGEHNLDVLEGNEQFLNAAK,3050000.0,1.525000e+05
2,SSGSSYPSLLQCLK,417000.0,2.978571e+04
3,SCTK,28200.0,7.050000e+03
4,LSSPATLNSR,17800000.0,1.780000e+06


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep302moddup = peaksdb302[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep302dup = peaksdb302[["stripped_peptide"]]

# deduplicate both of these lists
pep302mod = pep302moddup.drop_duplicates()
pep302 = pep302dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 302:", len(pep302moddup))
print("Deduplicated modified PeaksDB peptides in 302:", len(pep302mod))
print("Total stripped PeaksDB peptides in 302:", len(pep302dup))
print("Deduplicated stripped PeaksDB peptides in 302:", len(pep302))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep302.head()

Total modified PeaksDB peptides in 302: 62
Deduplicated modified PeaksDB peptides in 302: 62
Total stripped PeaksDB peptides in 302: 62
Deduplicated stripped PeaksDB peptides in 302: 59


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,SSGSSYPSLLQCLK
3,SCTK
4,LSSPATLNSR


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb302['A'].sum(),
        'C': peaksdb302['C'].sum(),
        'D': peaksdb302['D'].sum(),
        'E': peaksdb302['E'].sum(),
        'F': peaksdb302['F'].sum(),
        'G': peaksdb302['G'].sum(),
        'H': peaksdb302['H'].sum(),
        'I': peaksdb302['I'].sum(),
        'K': peaksdb302['K'].sum(),
        'L': peaksdb302['L'].sum(),
        'M': peaksdb302['M'].sum(),
        'N': peaksdb302['N'].sum(),
        'P': peaksdb302['P'].sum(),
        'Q': peaksdb302['Q'].sum(),
        'R': peaksdb302['R'].sum(),
        'S': peaksdb302['S'].sum(),
        'T': peaksdb302['T'].sum(),
        'V': peaksdb302['V'].sum(),
        'W': peaksdb302['W'].sum(),
        'Y': peaksdb302['Y'].sum(),
        'c-carb': peaksdb302['c-carb'].sum(),
        'm-oxid': peaksdb302['m-oxid'].sum(),
        'n-deam': peaksdb302['n-deam'].sum(),
        'q-deam': peaksdb302['q-deam'].sum(),
        'Total area': peaksdb302['Area'].sum(),
        'Total length': peaksdb302['stripped_length'].sum()
       }

totalpeaksdb302 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb302['% C w/ carb'] = totalpeaksdb302['c-carb'] / totalpeaksdb302['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb302['% M w/ oxid'] = totalpeaksdb302['m-oxid'] / totalpeaksdb302['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb302['% N w/ deam'] = totalpeaksdb302['n-deam'] / totalpeaksdb302['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb302['% Q w/ deam'] = totalpeaksdb302['q-deam'] / totalpeaksdb302['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb302['NAAF denom.'] = totalpeaksdb302['Total area'] / totalpeaksdb302['Total length']

# write modified dataframe to new txt file
totalpeaksdb302.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_totals.csv")

totalpeaksdb302.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,64,16,16,26,19,61,13,55,44,82,...,1,3,6,127130050.0,650,1.0,0.166667,0.068182,0.272727,195584.692308


In [10]:
# use the calculated NAAF factor (in totalpeaksdb302 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 195584.692308

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb302['NAAF factor'] = (peaksdb302['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb302_NAAF = peaksdb302[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb302_NAAF['A-NAAF20'] = peaksdb302_NAAF['A'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['C-NAAF20'] = peaksdb302_NAAF['C'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['D-NAAF20'] = peaksdb302_NAAF['D'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['E-NAAF20'] = peaksdb302_NAAF['E'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['F-NAAF20'] = peaksdb302_NAAF['F'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['G-NAAF20'] = peaksdb302_NAAF['G'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['H-NAAF20'] = peaksdb302_NAAF['H'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['I-NAAF20'] = peaksdb302_NAAF['I'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['K-NAAF20'] = peaksdb302_NAAF['K'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['L-NAAF20'] = peaksdb302_NAAF['L'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['M-NAAF20'] = peaksdb302_NAAF['M'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['N-NAAF20'] = peaksdb302_NAAF['N'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['P-NAAF20'] = peaksdb302_NAAF['P'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['Q-NAAF20'] = peaksdb302_NAAF['Q'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['R-NAAF20'] = peaksdb302_NAAF['R'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['S-NAAF20'] = peaksdb302_NAAF['S'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['T-NAAF20'] = peaksdb302_NAAF['T'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['V-NAAF20'] = peaksdb302_NAAF['V'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['W-NAAF20'] = peaksdb302_NAAF['W'] * peaksdb302['NAAF factor']
peaksdb302_NAAF['Y-NAAF20'] = peaksdb302_NAAF['Y'] * peaksdb302['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb302_NAAF['ccarb-NAAF20'] = peaksdb302_NAAF['c-carb'] * peaksdb302_NAAF['NAAF factor']
peaksdb302_NAAF['moxid-NAAF20'] = peaksdb302_NAAF['m-oxid'] * peaksdb302_NAAF['NAAF factor']
peaksdb302_NAAF['ndeam-NAAF20'] = peaksdb302_NAAF['n-deam'] * peaksdb302_NAAF['NAAF factor']
peaksdb302_NAAF['qdeam-NAAF20'] = peaksdb302_NAAF['q-deam'] * peaksdb302_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb302_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_naaf.csv")

peaksdb302_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,1.138777,3,2,0,1,0,3,0,1,...,0.000000,2.277553,2.277553,0.000000,1.138777,0.000000,2.277553,0.0,0.000000,0.0
1,LGEHNIDVLEGNEQFINAAK,0.779713,2,0,1,3,1,2,1,2,...,0.000000,0.000000,0.000000,0.779713,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,SSGSSYPSLLQCLK,0.152291,0,1,0,0,0,1,0,0,...,0.000000,0.761453,0.000000,0.000000,0.000000,0.152291,0.152291,0.0,0.000000,0.0
3,SCTK,0.036046,3,2,0,1,0,3,0,1,...,0.000000,0.072092,0.072092,0.000000,0.036046,0.000000,0.072092,0.0,0.036046,0.0
4,LSSPATLNSR,9.100917,1,0,0,0,0,0,0,0,...,9.100917,27.302750,9.100917,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [11]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb302 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb302_NAAF['NAAF factor'].sum(),
        'A': peaksdb302_NAAF['A-NAAF20'].sum(),
        'C': peaksdb302_NAAF['C-NAAF20'].sum(),
        'D': peaksdb302_NAAF['D-NAAF20'].sum(),
        'E': peaksdb302_NAAF['E-NAAF20'].sum(),
        'F': peaksdb302_NAAF['F-NAAF20'].sum(),
        'G': peaksdb302_NAAF['G-NAAF20'].sum(),
        'H': peaksdb302_NAAF['H-NAAF20'].sum(),
        'I': peaksdb302_NAAF['I-NAAF20'].sum(),
        'K': peaksdb302_NAAF['K-NAAF20'].sum(),
        'L': peaksdb302_NAAF['L-NAAF20'].sum(),
        'M': peaksdb302_NAAF['M-NAAF20'].sum(),
        'N': peaksdb302_NAAF['N-NAAF20'].sum(),
        'P': peaksdb302_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb302_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb302_NAAF['R-NAAF20'].sum(),
        'S': peaksdb302_NAAF['S-NAAF20'].sum(),
        'T': peaksdb302_NAAF['T-NAAF20'].sum(),
        'V': peaksdb302_NAAF['V-NAAF20'].sum(),
        'W': peaksdb302_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb302_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb302_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb302_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb302_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb302_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb302_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb302_NAAF['% C w/ carb.'] = totalpeaksdb302_NAAF['c-carb'] / totalpeaksdb302_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb302_NAAF['% M w/ oxid'] = totalpeaksdb302_NAAF['m-oxid'] / totalpeaksdb302_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb302_NAAF['% N w/ deam'] = totalpeaksdb302_NAAF['n-deam'] / totalpeaksdb302_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb302_NAAF['% Q w/ deam'] = totalpeaksdb302_NAAF['q-deam'] / totalpeaksdb302_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb302_NAAF['NAAF check'] = totalpeaksdb302_NAAF['NAAF'] / 195584.692308

# write modified dataframe to new txt file, same name + totals
totalpeaksdb302_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/302_BY_T24_GF_PDB/302B_BY_T24_GF_PDB_naaf_totals.csv")

totalpeaksdb302_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,72.846978,55.086994,5.205762,1.289753,4.476698,2.934618,15.351338,3.054672,12.31928,3.902546,...,5.563174,5.205762,0.003899,0.15671,0.172024,1.0,0.001829,0.007764,0.040431,0.000372
